<a href="https://colab.research.google.com/github/aya-bani/sgmd-mit-all-versions/blob/main/number_of_hidden_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np, pandas as pd, tensorflow as tf, random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from itertools import product


In [3]:
def set_seeds(seed=42):
    np.random.seed(seed); random.seed(seed); tf.random.set_seed(seed)


In [4]:
data = pd.read_csv("/content/drive/MyDrive/projet mit/data.csv", encoding="latin1")
X = data[["Tf_in","Ua","Uw"]].values
y = data["Y"].values.reshape(-1,1)

In [5]:
scaler_X = MinMaxScaler()
Xs = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler(feature_range=(0.2, 0.8))
ys = scaler_y.fit_transform(y)

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(Xs, ys, test_size=0.30, random_state=42)
X_val,   X_test, y_val,   y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)


In [7]:
def build_model(layers, hidden_act="sigmoid", out_act="sigmoid"):
    """
    layers: list of neuron counts, e.g. [12,8] = 2 hidden layers
    hidden_act: activation for hidden layers
    out_act: activation for output (sigmoid since Y in [0.2,0.8])
    """
    model = Sequential()
    # First hidden layer
    model.add(Dense(layers[0], input_dim=3, activation=hidden_act))
    # Additional hidden layers
    for n in layers[1:]:
        model.add(Dense(n, activation=hidden_act))
    # Output layer (1 neuron, regression)
    model.add(Dense(1, activation=out_act))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [8]:
def evaluate_model(model, X, y):
    """Return MSE, MAE, R² in original Y units."""
    y_pred = model.predict(X, verbose=0)
    y_true = scaler_y.inverse_transform(y)
    y_pred_inv = scaler_y.inverse_transform(y_pred)
    return (
        mean_squared_error(y_true, y_pred_inv),
        mean_absolute_error(y_true, y_pred_inv),
        r2_score(y_true, y_pred_inv)
    )

In [9]:
def sweep_layers(layer_grid, hidden_act="sigmoid"):
    results = []
    for layers in layer_grid:
        set_seeds(42); K.clear_session()
        model = build_model(layers, hidden_act=hidden_act)

        es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
        hist = model.fit(X_train, y_train,
                         validation_data=(X_val, y_val),
                         epochs=1000, batch_size=8, verbose=0,
                         callbacks=[es])

        # Metrics
        tr = evaluate_model(model, X_train, y_train)
        va = evaluate_model(model, X_val,   y_val)
        te = evaluate_model(model, X_test,  y_test)

        results.append({
            "layers": layers,
            "hidden_act": hidden_act,
            "train_mse": tr[0], "train_mae": tr[1], "train_r2": tr[2],
            "val_mse": va[0],   "val_mae": va[1],   "val_r2": va[2],
            "test_mse": te[0],  "test_mae": te[1],  "test_r2": te[2],
            "epochs_used": len(hist.history['loss'])
        })
    return pd.DataFrame(results)


In [10]:
# ============================
# 7. Define search grids
# ============================
grid_1 = [[n] for n in range(5, 21, 3)]                 # 1 hidden layer
grid_2 = [[n1,n2] for n1 in [20,15,10] for n2 in [15,10,5]]   # 2 hidden layers
grid_3 = [[15,10,5], [12,8,6], [20,15,10]]                    # 3 hidden layers
grid_4 = [[20,15,10,5], [15,12,8,5]]                          # 4 hidden layers

# ============================
# 8. Run sweeps (sigmoid hidden as baseline)
# ============================
print("Sweeping 1 hidden layer ...")
df1 = sweep_layers(grid_1, hidden_act="sigmoid")

print("\nSweeping 2 hidden layers ...")
df2 = sweep_layers(grid_2, hidden_act="sigmoid")

print("\nSweeping 3 hidden layers ...")
df3 = sweep_layers(grid_3, hidden_act="sigmoid")

print("\nSweeping 4 hidden layers ...")
df4 = sweep_layers(grid_4, hidden_act="sigmoid")

# Combine all
all_df = pd.concat([df1,df2,df3,df4], ignore_index=True)

# Best config
best_row = all_df.loc[all_df["val_mse"].idxmin()]
print("\n=== Best architecture (1–4 hidden layers) ===")
print(best_row[["layers","hidden_act","val_mse","val_mae","val_r2","test_mse","test_mae","test_r2"]])


Sweeping 1 hidden layer ...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo


Sweeping 2 hidden layers ...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo


Sweeping 3 hidden layers ...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Sweeping 4 hidden layers ...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



=== Best architecture (1–4 hidden layers) ===
layers         [10, 5]
hidden_act     sigmoid
val_mse       0.012836
val_mae       0.075279
val_r2        0.562207
test_mse      0.003474
test_mae      0.049921
test_r2        0.88916
Name: 14, dtype: object
